In [1]:
import numpy as np
from pyscipopt import Model, quicksum

In [2]:
model = Model("camiones")

In [3]:
# variables endogenas
costos = np.array([
    [9,6,5],
    [7,4,9],
    [4,6,3]
])

disponibles = [700, 600, 400] # A, B, C
necesitados = [800, 500, 400] # 1, 2, 3

cap_max = 600
costo_camion = 50
recargo_quinto_camion = 65 - 50 # 15

In [4]:
# variables exogenas

x = np.empty((3,3), dtype=object) # kilos transportados
y = np.empty((3,3), dtype=object) # camiones usados
for i in range(3):
    for j in range(3):
        x[i,j] = model.addVar(vtype="I", name=f"x_{i}_{j}", lb=0)
        y[i,j] = model.addVar(vtype="B", name=f"y_{i}_{j}")

quinto = model.addVar(vtype="B", name="quinto") # alquiler de quinto camion


In [5]:
# restricciones

# maxima carga de camiones
for i in range(3):
    for j in range(3):
        model.addCons(cap_max*y[i,j] >= x[i, j], name=f"capacidad maxima x_{i}_{j}")

# limite en cantidad de camiones
model.addCons(sum(y[i,j] for i in range(3) for j in range(3)) <= 4 + quinto, name="disponibilidad de camiones")

# el sitio 2 no puede recibir de A y B simultaneamente
model.addCons(y[0,1] + y[1,1] <= 1, name="sitio 2 no puede recibir de A y B simultaneamente")

# se satisfacen las necesidades de cada sitio
for j in range(3):
    model.addCons(sum(x[i, j] for i in range(3)) == necesitados[j])

# se obtienen los kilos disponibles
for i in range(3):
    model.addCons(sum(x[i, j] for j in range(3)) == disponibles[i])
    

In [6]:
# fo
model.setObjective(
    quicksum(costo_camion*y[i,j] + costos[i,j]*x[i,j] for i in range(3) for j in range(3)) + quinto*recargo_quinto_camion,
    sense="minimize"
    )

In [7]:
model.optimize()

sol = model.getBestSol()

model.writeSol(sol, "soluciones/ejercicio11.sol")